In [ ]:
import random
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
import numpy as np
from pyspark.sql.functions import count, avg, desc, round as round_func
from pyspark.sql.functions import col, lit, percentile_approx
from pyspark.sql.functions import when

In [ ]:
# Spark Session initialisieren
spark = SparkSession.builder \
    .appName("Yelp_Angriffsprofile Nuke") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "16g") \
    .getOrCreate()

# Lesen der JSON-Datei der Reviews
ratings = spark.read.json("yelp_academic_dataset_review.json")

In [ ]:
# Entfernen der nicht benötigten Spalten
ratings = ratings.drop('review_id')
ratings = ratings.drop('useful')
ratings = ratings.drop('funny')
ratings = ratings.drop('cool')
ratings = ratings.drop('text')
ratings = ratings.drop('date')

In [ ]:
# Lesen der JSON-Datei der Unternehmen
businesses = spark.read.json("yelp_academic_dataset_business.json")

In [ ]:
# Anzeigen der ersten 10 Ratings
ratings.show(10)

In [ ]:
# Anzeigen der ersten 10 Unternehmen
businesses.show(10)

In [ ]:
# Entfernen der nicht benötigten Spalten
businesses = businesses.drop('address')
businesses = businesses.drop('attributes')
businesses = businesses.drop('categories')
businesses = businesses.drop('city')
businesses = businesses.drop('hours')
businesses = businesses.drop('is_open')
businesses = businesses.drop('latitude')
businesses = businesses.drop('longitude')
businesses = businesses.drop('name')
businesses = businesses.drop('postal_code')
businesses = businesses.drop('review_count')
businesses = businesses.drop('stars')

In [ ]:
# Verbinden der DataFrames basierend auf der businessId
ratings = ratings.join(businesses, on='business_id', how='inner')

In [ ]:
# Umbenennen der Spalten in "userId", "businessId" und "rating"
businesses = businesses.withColumnRenamed("business_id", "businessId")
ratings = ratings.withColumnRenamed("user_id", "userId") \
                 .withColumnRenamed("business_id", "businessId") \
                 .withColumnRenamed("stars", "rating")

In [ ]:
# Anzeigen der ersten 10 Ratings
ratings.show(10)

In [ ]:
# Anzeigen des Schemas und der ersten 10 Ratings
ratings.printSchema()
ratings.show(10)

In [ ]:
# Anzahl der Ratings im Datensatz
print("Anzahl der Ratings im Datensatz:", ratings.count())

# Duplikate filtern
duplicates = (
    ratings.groupBy("userId", "businessId")
    .count()
    .filter(F.col("count") > 1)
)

# Duplikate anzeigen
print("Anzahl der Duplikate:", duplicates.count())

# Duplikate entfernen
ratings = ratings.dropDuplicates(["userId", "businessId"])

# Ergebnis überprüfen: Anzahl der Ratings im Datensatz nach Entfernung der Duplikate
print("Anzahl der Zeilen nach Entfernen der Duplikate:")
print(ratings.count())

In [ ]:
# Berechne den Mittelwert und die Standardabweichung der Ratings im Datensatz und zeige diese an
mean_rating_system = ratings.agg({"rating": "avg"}).collect()[0][0]
stddev_rating = ratings.agg({"rating": "stddev"}).collect()[0][0]

print(mean_rating_system)
print(stddev_rating)

In [ ]:
# Anzahl aller Ratings im DataFrame
total_ratings_counter = ratings.count()

# Ausgabe der Anzahl der Ratings
print("Anzahl aller Ratings:", total_ratings_counter)

In [ ]:
# Berechnung der Beliebtheit und der durchschnittlichen Bewertung pro Unternehmen
item_popularity_and_avg_rating = (
    ratings.groupBy("businessId")
    .agg(
        F.count("rating").alias("num_ratings"),
        F.avg("rating").alias("avg_rating")
    )
)

# Finde das mittlere Quantil für die Anzahl der Bewertungen
quantiles = item_popularity_and_avg_rating.approxQuantile("num_ratings", [0.4, 0.6], 0.05)
min_num_ratings, max_num_ratings = quantiles[0], quantiles[1]

# Mittlere Bewertung: Finde das mittlere Quantil für die durchschnittliche Bewertung
rating_quantiles = item_popularity_and_avg_rating.approxQuantile("avg_rating", [0.4, 0.6], 0.05)
min_avg_rating, max_avg_rating = rating_quantiles[0], rating_quantiles[1]

# Wähle repräsentativen Bundesstaat
popular_states = (
    businesses
    .withColumn("state", F.explode(F.split(F.col("state"), "\|")))
    .groupBy("state")
    .count()
    .orderBy(F.desc("count"))
    .limit(3)
)

# Filtere die Unternehmen mit mittlerer Beliebtheit, mittlerer Bewertung und einem repräsentativen Bundesstaat
target_candidates = (
    item_popularity_and_avg_rating
    .filter(
        (F.col("num_ratings") >= min_num_ratings) & (F.col("num_ratings") <= max_num_ratings) &
        (F.col("avg_rating") >= min_avg_rating) & (F.col("avg_rating") <= max_avg_rating)
    )
    .join(businesses, "businessId")
    .withColumn("state", F.explode(F.split(F.col("state"), "\|")))
    .join(popular_states, "state")
)

# Zeige ein paar Kandidaten, die als Target-Item verwendet weden könnten
target_candidates.show(5, truncate=False)

In [ ]:
# Filtere den DataFrame so, dass nur businessIds mit einem nicht-leeren Bundesstaat vorhanden sind
filtered_states = ratings.filter(ratings["state"].isNotNull() & (ratings["state"] != ""))

# Liste aller verfügbaren businessIds aus dem Datensatz abrufen
all_business_ids = ratings.select("businessId").distinct().rdd.flatMap(lambda x: x).collect()

# Funktion zum Validieren der eingegebenen businessId
def get_valid_target_item():
    while True:
        user_input = input("Bitte gib die businessId des Ziel-Items ein oder tippe 'zufällig' für ein zufälliges Item: ")
        
        if user_input.lower() == 'zufällig':
            # Wähle ein zufälliges gültiges Item aus der Liste
            return random.choice(all_business_ids)
        
        try:
            # Nutzeraufforderung in eine Zahl umwandeln
            target_item = str(user_input)
            
            # Überprüfen, ob die eingegebene businessID im Datensatz vorhanden ist
            if target_item in all_business_ids:
                return target_item
            else:
                print("Die eingegebene businessId ist ungültig. Bitte versuche es erneut.")
        except ValueError:
            print("Ungültige Eingabe! Bitte gib eine gültige businessId ein oder tippe 'zufällig'.")

# Abrufen der validierten businessID vom Nutzer
target_item = get_valid_target_item()

print(f"Das Ziel-Item (businessId) für den Angriff lautet: {target_item}")

In [ ]:
# Gesamtanzahl der User Profile im Datensatz
total_user_profiles = ratings.select("userId").distinct().count()

# Aufforderung, einen %-Anteil für die Anzahl der Angriffsprofile einzugeben
percentage_of_attacks = float(input("Bitte gib den Prozentsatz der Angriffsprofile ein (z.B. 5 für 5%): "))

# Berechne die Anzahl der Angriffsprofile basierend auf dem eingegebenen Prozentsatz
num_attack_profiles = int((percentage_of_attacks / 100) * total_user_profiles)

print(f"Die Anzahl der Angriffsprofile beträgt: {num_attack_profiles}")

In [ ]:
# Berechnung der Durchschnittsbewertungen pro businessID
item_avg_ratings = ratings.groupBy("businessId").agg(avg("rating").alias("avg_rating"))
item_avg_ratings.show(10)
# Konvertiere item_avg_ratings in ein Dictionary für schnellen Zugriff
item_avg_ratings_dict = {row["businessId"]: row["avg_rating"] for row in item_avg_ratings.collect()}

In [ ]:
# Berechnung des Schwellenwerts für das untere Quartil der Durchschnittsbewertungen
lower_quartile = item_avg_ratings.approxQuantile("avg_rating", [0.25], 0.01)[0]  # 25. Perzentil

# Filtere Items nach dem unteren Quartil und sortiere nach Anzahl der Bewertungen
unpopular_items_negative = ratings.join(item_avg_ratings, "businessId") \
    .filter((col("avg_rating") <= lit(lower_quartile))) \
    .groupBy("businessId") \
    .agg(count("*").alias("rating_count")) \
    .orderBy(col("rating_count").asc()) \
    .limit(1)

# Konvertiere das Ergebnis in eine Liste, wobei das Target-Item ausgeschlossen wird
unpopular_items_list = [
    row["businessId"] for row in unpopular_items_negative.collect() if row["businessId"] != target_item
]

print("Top 1 unpopuläres Unternehmen (businessId), basierend auf unteren Quartil der Bewertungen, ohne das Target-Item:", unpopular_items_list)

In [ ]:
# Filtere Zeilen, die einen nicht-leeren Bundesstaat haben
businesses_with_states_df = ratings.filter(F.col("state").isNotNull() & (F.col("state") != "")).select("businessId", "state")

# Funktion zum Aufteilen der Bundesstaaten und Erstellen eines Paares (state, businessId)
def split_states(row):
    state_id = row["businessId"]
    states = row["state"].split("|")
    return [(state, state_id) for state in states]

# RDD erstellen, indem die Unternehmen aufgeteilt werden und dann nach Bundesstaat gruppiert werden
businesses_by_state_rdd = businesses_with_states_df.rdd.flatMap(split_states).groupByKey()

# Konvertiere das RDD in ein Dictionary, um schnellen Zugriff auf businessIds nach Bundesstaat zu ermöglichen
businesses_by_state = businesses_by_state_rdd.mapValues(list).collectAsMap()

# Ziel-Item abfragen
target_item_states = businesses_with_states_df.filter(F.col("businessId") == target_item).select("state").collect()[0]["state"].split('|')

# Liste aller Kandidaten für Segment-Items erstellen
segment_businesses_candidates = []

for state in target_item_states:
    if state in businesses_by_state:
        for state in businesses_by_state[state]:
            # Nur hinzufügen, wenn das Segment-Item nicht das Target-Item ist und nicht bereits vorhanden ist
            if state != target_item and state not in segment_businesses_candidates:
                segment_businesses_candidates.append(state)

# Zufällige Auswahl von einem Segment-Item aus den Kandidaten
segment_businesses = random.sample(segment_businesses_candidates, 1) if len(segment_businesses_candidates) > 1 else segment_businesses_candidates

print(segment_businesses_candidates)

# Ausgabe des ausgewählten Segment-Items
print(segment_businesses)

In [ ]:
# Funktion zur Generierung von Filler-Ratings nach Normalverteilung
def generate_filler_rating(mean_rating):
    # Generiere eine Bewertung nach der Normalverteilung basierend auf dem übergebenen Mittelwert
    rating = np.random.normal(mean_rating, stddev_rating)
    # Begrenze das Rating auf die Skala von 1 bis 5
    rating = max(1, min(5, rating))
    # Runde das Rating auf die nächste ganze Zahl (1, 2, ..., 5)
    return round(rating)

In [ ]:
# Angriffstyp auswählen und validieren
attack_type = input("Wähle Angriffstyp ('reverse_bandwagon', 'random_nuke', 'average_nuke' oder 'segment_nuke'): ").lower()
while attack_type not in ['reverse_bandwagon', 'random_nuke', 'average_nuke', 'segment_nuke']:
    print("Ungültige Eingabe! Bitte wähle entweder 'reverse_bandwagon', 'random_nuke', 'average_nuke' oder 'segment_nuke'.")
    attack_type = input("Wähle Angriffstyp ('reverse_bandwagon', 'random_nuke', 'average_nuke' oder 'segment_nuke'): ").lower()

In [ ]:
# Angriffsart anzeigen
print(attack_type)

In [ ]:
# Gruppiere nach 'userId' und zähle die Ratings
ratings_per_user = ratings.groupBy("userId").agg(count("rating").alias("num_ratings"))

# Berechne den Durchschnitt der Ratings pro Benutzer
average_ratings_per_user = ratings_per_user.agg(avg("num_ratings")).collect()[0][0]

# Runde den Durchschnitt und ziehe 1 für das Target-Item ab
rounded_average = round(average_ratings_per_user - 1)

# Ausgabe der durchschnittlichen Anzahl an Ratings pro Profil abzüglich von 1
print("Durchschnittliche Anzahl an Ratings pro Profil (gerundet, nach Abzug von 1):", rounded_average)

In [ ]:
# Funktion zur Generierung einer Profilgröße basierend auf einer Normalverteilung
def generate_attack_profile_size(average):
    # Generiere eine Anzahl von Bewertungen basierend auf einer Normalverteilung
    std_dev = 0.5 * average  # Standardabweichung = 50% des Durchschnitts
    profile_size = int(np.random.normal(average, std_dev))

    # Stelle sicher, dass die Profillänge mindestens 1 ist
    profile_size = max(profile_size, 1)
    
    return profile_size

In [ ]:
# Funktion zum Erstellen von Angriffs-Bewertungen für jeden neuen User
def generate_attack_ratings(user_id, target_item, unpopular_items_list, all_business_ids, attack_type, segment_businesses):
    ratings_list = []
    
    # Ziel-Item mit der minimalen Bewertung (1) bewerten
    ratings_list.append((user_id, target_item, 1))
    
    # Bestimme die Anzahl der Filler-Items basierend auf der Normalverteilung
    num_fillers = generate_attack_profile_size(rounded_average)
    
    # Reduziere die Anzahl der Filler-Items bei Reverse Bandwagon und Segment Nuke-Angriffen
    if attack_type == 'reverse_bandwagon' or attack_type == 'segment_nuke':
        num_fillers = max(num_fillers - 1, 0)  # Setze Untergrenze auf 0
    
    # Füge unpopuläre Items hinzu bei Reverse Bandwagon
    if attack_type == 'reverse_bandwagon':
        # 1 unpopulärstes Item mit 1 bewerten
        for businessId in unpopular_items_list:
            ratings_list.append((user_id, businessId, 1))
    
    # Füge Segment-Items hinzu bei Segment Nuke-Angriff
    elif attack_type == 'segment_nuke' and segment_businesses:
        # 1 Item aus der vorbereiteten Segment-Liste bewerten
        for businessId in segment_businesses:
            ratings_list.append((user_id, businessId, 1))

    # Liste der nicht erlaubten Items (immer das Target-Item)
    not_allowed_items = set([target_item])  # Set für Effizienz

    # Füge bei Reverse Bandwagon-Angriff die unpopulären Items hinzu
    if attack_type == 'reverse_bandwagon':
        not_allowed_items.update(unpopular_items_list)
    
    # Füge bei Segment Nuke-Angriff die Segment-Items hinzu
    if attack_type == 'segment_nuke' and segment_businesses:
        not_allowed_items.update(segment_businesses)
    
    # Filtere die Filler-Items, sodass sie keine "nicht erlaubten" Items enthalten
    available_filler_items = [businessId for businessId in all_business_ids if businessId not in not_allowed_items]

    # Wähle zufällige Filler-Items aus den verbleibenden Businesses aus
    random_states = random.sample(available_filler_items, num_fillers)
    
    # Filler-Items bewerten
    for businessId in random_states:
        if attack_type == 'average_nuke':
            mean_rating = item_avg_ratings_dict.get(businessId, mean_rating_system)
            filler_rating = generate_filler_rating(mean_rating)
        elif attack_type == 'segment_nuke':
            filler_rating = 5  # Bei Segment Nuke-Angriffen werden Filler-Items immer mit der maximalen Bewertung (5) bewertet
        else:
            filler_rating = generate_filler_rating(mean_rating_system)

        ratings_list.append((user_id, businessId, filler_rating))
    
    return ratings_list

# Liste aller verfügbaren Unternehmen
all_businesses_ids = ratings.select("businessId").distinct().rdd.flatMap(lambda x: x).collect()

# Generiere Angriffs-Bewertungen und speichere sie als DataFrame
attack_ratings = []
for i in range(num_attack_profiles):
    user_id = 100000000 + i  # Erstelle eine neue UserID für jeden Angriff (angenommen IDs >= 100000000 sind neu)
    
    # Erzeuge Angriffsbewertungen
    attack_ratings.extend(generate_attack_ratings(
        user_id, 
        target_item, 
        unpopular_items_list, 
        all_business_ids, 
        attack_type, 
        segment_businesses
    ))

# Erstelle ein DataFrame mit den generierten Angriffsbewertungen
attack_ratings_df = spark.createDataFrame(attack_ratings, ["userId", "businessId", "rating"])

# Zeige die ersten 10 Einträge des Angriffs-Dataframes
attack_ratings_df.show(10)

In [ ]:
# Anzahl der Einträge im attack_ratings_df
num_entries = attack_ratings_df.count()
print(f"Anzahl der Einträge (Bewertungen) in Angriffsratings: {num_entries}")

In [ ]:
# Angriffsart anzeigen
print(attack_type)

In [ ]:
# Anzahl verschiedener userIds
num_users = attack_ratings_df.select("userId").distinct().count()
print(f"Anzahl verschiedener User IDs in Angriffsratings: {num_users}")

In [ ]:
# Anzahl verschiedener businessIds
num_states = attack_ratings_df.select("businessId").distinct().count()
print(f"Anzahl verschiedener businessIds: {num_states}")

In [ ]:
# Häufigkeit der Ratings im Angriffs-Dataframe
rating_counts = attack_ratings_df.groupBy("rating").count().orderBy("rating")
rating_counts.show()

# Alternativ als Python-Dictionary
rating_counts_dict = {row['rating']: row['count'] for row in rating_counts.collect()}
print(f"Häufigkeit der Bewertungen in Angriffsratings: {rating_counts_dict}")

In [ ]:
# Nur relevante Spalten aus dem ursprünglichen ratings DataFrame auswählen
ratings = ratings.select("userId", "businessId", "rating")

In [ ]:
# Ursprüngliche Daten und Angriffs-Daten kombinieren
all_ratings = ratings.union(attack_ratings_df)
all_ratings.show(10)

In [ ]:
# Häufigkeit der Ratings im Gesamt-Dataframe
rating_counts2 = all_ratings.groupBy("rating").count().orderBy("rating")
rating_counts2.show()

In [ ]:
# Label-Spalte hinzufügen: 0 für normale User (vorhandene), 1 für Angriffs-User
all_ratings = all_ratings.withColumn("Label", 
                                     when(col("userId") >= 100000000, 1)
                                     .otherwise(0))

# Überprüfen des finalen Datensatzes
all_ratings.show(10)

In [ ]:
# Variable mit Namen des Datensatzes
datensatz = "Yelp"

In [ ]:
# Dynamischer Dateiname basierend auf Datensatz, Angriffsart und Angriffsgröße
dateiname = f"all_ratings_{datensatz}_{attack_type}_{percentage_of_attacks}.csv"

# Speichern des DataFrames mit dem dynamischen Dateinamen
all_ratings.coalesce(1).write.csv(dateiname, header=True)

In [ ]:
# Filter auf userId >= 100000000 (Angreifer)
filtered_ratings = all_ratings.filter(col("userId") >= 100000000)

# Anzeigen der gefilterten Zeilen
filtered_ratings.show(10)

In [ ]:
# Filter auf userId >= 100000000
filtered_ratings2 = all_ratings.filter(col("userId") == 100000000)

# Anzeigen der Bewertungen von Nutzer mit userId 100000000
filtered_ratings2.show(10)

# Häufigkeit der Ratings bei User 100000000
rating_counts2 = filtered_ratings2.groupBy("rating").count().orderBy("rating")
rating_counts2.show()

# Alternativ als Python-Dictionary
rating_counts_dict2 = {row['rating']: row['count'] for row in rating_counts2.collect()}
print(f"Häufigkeit der Bewertungen: {rating_counts_dict2}")

In [ ]:
# Berechne den Durchschnitt der Bewertungen für userId 100000000
average_rating_user = attack_ratings_df.filter(col("userId") == 100000000).agg(avg("rating")).collect()[0][0]

print(f"Der Durchschnitt der Bewertungen von User 100000000 ist: {average_rating_user}")

In [ ]:
# Berechne die neue durchschnittliche Bewertung im Gesamt-Dataframe nach einem Angriff
mean_rating2 = all_ratings.agg({"rating": "avg"}).collect()[0][0]
print(mean_rating2)

In [ ]:
# Anzahl aller Ratings im Angriffs-DataFrame
total_ratings_count_attack = attack_ratings_df.count()

# Ausgabe der Anzahl der Ratings
print("Anzahl aller Angriffsratings:", total_ratings_count_attack)

In [ ]:
# Anzahl aller Ratings im Gesamt-Dataframe
total_ratings_count = all_ratings.count()

# Ausgabe der Anzahl der Ratings
print("Anzahl aller Ratings:", total_ratings_count)

In [ ]:
# Definiere die User-IDs für die Abfrage
user_ids_mio = [i for i in range(100000000, 100000021)]  # User von 100000000 bis 100000020

# Filtere den Datensatz für die gewünschten User und zähle die Ratings pro User
ratings_count_mio = all_ratings.filter(F.col("userId").isin(user_ids_mio)) \
                       .groupBy("userId") \
                       .agg(F.count("rating").alias("num_ratings")) \
                       .orderBy("userId")

# Zeige die Anzahl der Ratings für die angegebenen User
ratings_count_mio.show()

In [ ]:
# Beende die Spark-Session
spark.stop()